### Fine-tuning roberta large on Squad dataset for question-answering using adapter-lora concept -Hemant


In [1]:
!pip install -U adapter-transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 45.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 33.7 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0


In [2]:
from datasets import load_dataset, load_metric
raw_datasets = load_dataset('squad_v2')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [3]:
from transformers.adapters import LoRAConfig

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:

do_train = True # False
do_eval = True

# epochs, bs, GA
evaluation_strategy = "epoch" # no

# fp16
#fp16_opt_level = 'O1'
#fp16_backend = "auto"
#fp16_full_eval = False

# optimizer (AdamW)
weight_decay = 0.01 # 0.0
adam_beta1 = 0.9
adam_beta2 = 0.999

# scheduler
lr_scheduler_type = 'linear'
warmup_ratio = 0.0
warmup_steps = 0

# logs
logging_strategy = "steps"
logging_first_step = True # False
logging_steps = 500     # if strategy = "steps"
eval_steps = logging_steps # logging_steps

# checkpoints
save_strategy = "epoch" # steps
save_steps = 1000 # if save_strategy = "steps"
save_total_limit = 1 # None

# no cuda, seed
no_cuda = False
seed = 42

# bar
disable_tqdm = False # True
remove_unused_columns = True
path_to_outputs = "./outputs"

# subfolder for model outputs
output_dir = path_to_outputs + '/output_dir'
overwrite_output_dir = True # False

# logs
logging_dir = path_to_outputs + '/logging_dir'
batch_size = 8#16
gradient_accumulation_steps = 1

learning_rate = 1e-4
num_train_epochs = 3

adam_epsilon = 1e-7

fp16 = False

# best model
load_best_model_at_end = True
metric_for_best_model = "loss"
greater_is_better = False

In [5]:
from transformers import AutoTokenizer, RobertaTokenizerFast

In [6]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-large")

In [7]:
max_length = 300 # The maximum length of a feature (question and context)
doc_stride = 128

In [8]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


In [9]:
pad_on_right = tokenizer.padding_side == "right"
tokenized_datasets = raw_datasets.map(prepare_train_features, batched=True, remove_columns=raw_datasets["train"].column_names)

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [10]:

from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("roberta-large")

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForQuestionAnswering: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-large and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and i

In [12]:
 for name, p in model.named_parameters():
   if p.requires_grad == True:
     print(name)

roberta.embeddings.word_embeddings.weight
roberta.embeddings.position_embeddings.weight
roberta.embeddings.token_type_embeddings.weight
roberta.embeddings.LayerNorm.weight
roberta.embeddings.LayerNorm.bias
roberta.encoder.layer.0.attention.self.query.weight
roberta.encoder.layer.0.attention.self.query.bias
roberta.encoder.layer.0.attention.self.key.weight
roberta.encoder.layer.0.attention.self.key.bias
roberta.encoder.layer.0.attention.self.value.weight
roberta.encoder.layer.0.attention.self.value.bias
roberta.encoder.layer.0.attention.output.dense.weight
roberta.encoder.layer.0.attention.output.dense.bias
roberta.encoder.layer.0.attention.output.LayerNorm.weight
roberta.encoder.layer.0.attention.output.LayerNorm.bias
roberta.encoder.layer.0.intermediate.dense.weight
roberta.encoder.layer.0.intermediate.dense.bias
roberta.encoder.layer.0.output.dense.weight
roberta.encoder.layer.0.output.dense.bias
roberta.encoder.layer.0.output.LayerNorm.weight
roberta.encoder.layer.0.output.LayerNorm

In [13]:
config = LoRAConfig(r=8, alpha=16)
model.add_adapter("lora_adapter", config=config)
model.merge_adapter("lora_adapter")
model.reset_adapter()

In [14]:
model.train_adapter(["lora_adapter"])
model.set_active_adapters("lora_adapter")

In [16]:

for name, p in model.named_parameters():
  if p.requires_grad == True:
    print(name)

roberta.encoder.layer.0.attention.self.query.loras.lora_adapter.lora_A
roberta.encoder.layer.0.attention.self.query.loras.lora_adapter.lora_B
roberta.encoder.layer.0.attention.self.value.loras.lora_adapter.lora_A
roberta.encoder.layer.0.attention.self.value.loras.lora_adapter.lora_B
roberta.encoder.layer.1.attention.self.query.loras.lora_adapter.lora_A
roberta.encoder.layer.1.attention.self.query.loras.lora_adapter.lora_B
roberta.encoder.layer.1.attention.self.value.loras.lora_adapter.lora_A
roberta.encoder.layer.1.attention.self.value.loras.lora_adapter.lora_B
roberta.encoder.layer.2.attention.self.query.loras.lora_adapter.lora_A
roberta.encoder.layer.2.attention.self.query.loras.lora_adapter.lora_B
roberta.encoder.layer.2.attention.self.value.loras.lora_adapter.lora_A
roberta.encoder.layer.2.attention.self.value.loras.lora_adapter.lora_B
roberta.encoder.layer.3.attention.self.query.loras.lora_adapter.lora_A
roberta.encoder.layer.3.attention.self.query.loras.lora_adapter.lora_B
robert

In [17]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction
#from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from datasets import load_metric

training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=overwrite_output_dir,
        do_train=do_train,
        do_eval=do_eval,
        evaluation_strategy=evaluation_strategy,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        adam_beta1=adam_beta1,
        adam_beta2=adam_beta2,
        adam_epsilon=adam_epsilon,
        num_train_epochs=num_train_epochs,
        lr_scheduler_type=lr_scheduler_type,
        warmup_ratio=warmup_ratio,
        warmup_steps=warmup_steps,
        logging_dir=logging_dir,         # directory for storing logs
        logging_strategy=evaluation_strategy,
        logging_steps=logging_steps,     # if strategy = "steps"
        save_strategy=evaluation_strategy,          # model checkpoint saving strategy
        save_steps=logging_steps,        # if strategy = "steps"
        save_total_limit=save_total_limit,
        fp16=fp16,
        eval_steps=logging_steps,        # if strategy = "steps"
        load_best_model_at_end=load_best_model_at_end,
        metric_for_best_model=metric_for_best_model,
        greater_is_better=greater_is_better
        )


In [18]:
from transformers.trainer_callback import EarlyStoppingCallback
from transformers import default_data_collator

data_collator = default_data_collator
train_adapter=True
#do_save_full_model=train_adapter, # save full model as we finetuned head + embeddings
trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
    )

In [19]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 136788
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 25650
  Number of trainable parameters = 788482
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,1.188200,0.787869
2,0.832200,0.768669
3,0.761900,0.713840


***** Running Evaluation *****
  Num examples = 12711
  Batch size = 16
Saving model checkpoint to ./outputs/output_dir/checkpoint-8550
Configuration saved in ./outputs/output_dir/checkpoint-8550/lora_adapter/adapter_config.json
Module weights saved in ./outputs/output_dir/checkpoint-8550/lora_adapter/pytorch_adapter.bin
Configuration saved in ./outputs/output_dir/checkpoint-8550/lora_adapter/head_config.json
Module weights saved in ./outputs/output_dir/checkpoint-8550/lora_adapter/pytorch_model_head.bin
tokenizer config file saved in ./outputs/output_dir/checkpoint-8550/tokenizer_config.json
Special tokens file saved in ./outputs/output_dir/checkpoint-8550/special_tokens_map.json
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num

TrainOutput(global_step=25650, training_loss=0.9274202759502924, metrics={'train_runtime': 38870.5222, 'train_samples_per_second': 10.557, 'train_steps_per_second': 0.66, 'total_flos': 2.238863216864112e+17, 'train_loss': 0.9274202759502924, 'epoch': 3.0})

In [20]:
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples


In [21]:
validation_features = raw_datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names
)


Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [22]:

raw_predictions = trainer.predict(validation_features)

The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 12711
  Batch size = 16


In [23]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))


In [24]:
max_answer_length = 30

In [25]:
from tqdm.auto import tqdm
import collections

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        #if not squad_v2:
        #    predictions[example["id"]] = best_answer["text"]
        #else:
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions


In [26]:
final_predictions = postprocess_qa_predictions(raw_datasets["validation"], validation_features, raw_predictions.predictions)


Post-processing 11873 example predictions split into 12711 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

In [27]:
metric = load_metric("squad_v2")
formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in raw_datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)


/tmp/ipykernel_29/3461119883.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2")


{'exact': 83.07925545355008,
 'f1': 86.27646741610619,
 'total': 11873,
 'HasAns_exact': 81.5114709851552,
 'HasAns_f1': 87.91506370300765,
 'HasAns_total': 5928,
 'NoAns_exact': 84.64255677039529,
 'NoAns_f1': 84.64255677039529,
 'NoAns_total': 5945,
 'best_exact': 83.02872062663185,
 'best_exact_thresh': 0.0,
 'best_f1': 86.22593258918776,
 'best_f1_thresh': 0.0}

In [28]:
import json
pred_file = open("/kaggle/working/predictions_2.json","w")
predictions= metric.compute(predictions=formatted_predictions,references=references)
json.dump(final_predictions,pred_file)

In [29]:
trainer.model.save_adapter("/kaggle/working/", "lora_adapter")
trainer.model.save_all_adapters("/kaggle/working/")

Configuration saved in /kaggle/working/adapter_config.json
Module weights saved in /kaggle/working/pytorch_adapter.bin
Configuration saved in /kaggle/working/head_config.json
Module weights saved in /kaggle/working/pytorch_model_head.bin
Configuration saved in /kaggle/working/lora_adapter/adapter_config.json
Module weights saved in /kaggle/working/lora_adapter/pytorch_adapter.bin
Configuration saved in /kaggle/working/lora_adapter/head_config.json
Module weights saved in /kaggle/working/lora_adapter/pytorch_model_head.bin


In [30]:
trainer.save_model("/kaggle/working/")

Saving model checkpoint to /kaggle/working/
Configuration saved in /kaggle/working/lora_adapter/adapter_config.json
Module weights saved in /kaggle/working/lora_adapter/pytorch_adapter.bin
Configuration saved in /kaggle/working/lora_adapter/head_config.json
Module weights saved in /kaggle/working/lora_adapter/pytorch_model_head.bin
tokenizer config file saved in /kaggle/working/tokenizer_config.json
Special tokens file saved in /kaggle/working/special_tokens_map.json
